In [7]:
import os,time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import random

In [10]:
file_url = 'https://raw.githubusercontent.com/ashutoshpandey5891/NLP/master/spell_correction/data/wikipedia.txt'

import requests
file_pth = 'wikipedia.txt'
with open(file_pth,'w') as f:
  data = requests.get(file_url).content.decode()
  f.write(data)

In [11]:
data = []
with open(file_pth) as f:
    for line in f.readlines():
        cor,incor = line.strip().split(':')
        for c in cor.strip().split():
            for inc in incor.strip().split():
                data.append((inc,c))
print(len(data))
print(data[:5])

2455
[('Apenines', 'Apennines'), ('Appenines', 'Apennines'), ('Athenean', 'Athenian'), ('Atheneans', 'Athenians'), ('Bernouilli', 'Bernoulli')]


In [16]:
##### utils

def check_accuracy(correct_func,name):
    start_time = time.time()
    print('Testing Accuracy for : {}'.format(name))
    cor_count = 0
    tot_count = len(data)
    incor_data = []
    for inc,c in tqdm(data):
        corrected = correct_func(inc)
        if corrected == c:
            cor_count += 1
        else:
            incor_data.append((inc,c,corrected))
    print('Model Accuracy : {:.4f}'.format(cor_count/tot_count))
    print('Sample Incorrections\nIncorrect:Correct:Corrected')
    for i in range(10):
        idx = random.choice(range(len(incor_data)))
        print('{} : {} : {}'.format(incor_data[idx][0],incor_data[idx][1],incor_data[idx][2]))
    time_taken = time.time()-start_time
    return incor_data,cor_count/tot_count,time_taken

In [17]:
all_results = {}

In [13]:
# !!sudo apt-get install swig3.0

In [2]:
!sudo pip install --upgrade jamspell

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!wget https://github.com/bakwc/JamSpell-models/raw/master/en.tar.gz
!tar -xvf en.tar.gz

--2023-02-17 17:49:09--  https://github.com/bakwc/JamSpell-models/raw/master/en.tar.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/bakwc/JamSpell-models/master/en.tar.gz [following]
--2023-02-17 17:49:09--  https://raw.githubusercontent.com/bakwc/JamSpell-models/master/en.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36611828 (35M) [application/octet-stream]
Saving to: ‘en.tar.gz.3’

en.tar.gz.3         100%[===================>]  34.92M  --.-KB/s    in 0.1s    

2023-02-17 17:49:09 (264 MB/s) - ‘en.tar.gz.3’ saved [36611828/36611828]

en.bin


In [14]:
import jamspell

name = 'JamSpell'
corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('en.bin')

def predict_correct_jamspell(inc):
    return corrector.FixFragment(inc)

In [18]:
name = 'JamSpell'
incor_data,acc,tt = check_accuracy(predict_correct_jamspell,name)
all_results[name] = {'accuracy':acc,'time_taken':tt}

Testing Accuracy for : JamSpell


100%|██████████| 2455/2455 [00:01<00:00, 1405.57it/s]

Model Accuracy : 0.6676
Sample Incorrections
Incorrect:Correct:Corrected
irelevent : irrelevant : relevant
exerbate : exacerbate : generate
afficionado : aficionado : afficionado
reccomended : recommended : reccomended
imigrant : emigrant : migrant
fomr : from : for
protruberance : protuberance : protruberance
pronounciation : pronunciation : pronounciation
quizes : quizzes : quizes
Newyorker : New_Yorker : Newyorker


In [21]:
from textblob import TextBlob

def predict_correct_textblob(inc):
    return TextBlob(inc).correct()

name = 'TextBlob'
incor_data,acc,tt = check_accuracy(predict_correct_textblob,name)
all_results[name] = {'accuracy':acc,'time_taken':tt}

Testing Accuracy for : TextBlob


100%|██████████| 2455/2455 [04:11<00:00,  9.76it/s]

Model Accuracy : 0.6167
Sample Incorrections
Incorrect:Correct:Corrected
Palistinian : Palestinian : Palistinian
proceding : proceeding : preceding
archeaologists : archaeologists : archeaologists
forunner : forerunner : founder
enviroments : environments : environment
clera : sclera : clear
erested : erected : rested
comited : committed : vomited
basicly : basically : easily
mear : wear : dear


In [22]:
!pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 39.9 MB/s eta 0:00:00


In [23]:
from spellchecker import SpellChecker

sp_checker = SpellChecker('en')

def predict_correct_spellchecker(inc):
    return sp_checker.correction(inc)

name = 'PySpellChecker'
incor_data,acc,tt = check_accuracy(predict_correct_spellchecker,name)
all_results[name] = {'accuracy':acc,'time_taken':tt}

Testing Accuracy for : PySpellChecker


100%|██████████| 2455/2455 [01:58<00:00, 20.79it/s] 

Model Accuracy : 0.7255
Sample Incorrections
Incorrect:Correct:Corrected
ther : their : the
thru : through : thou
accension : accession : ascension
blaimed : blamed : claimed
desparate : disparate : desperate
litature : literature : ligature
phenomenonly : phenomenally : phenomenon
gae : Gael : gave
tremelo : tremolo : tremble
alusion : illusion : allusion


In [24]:
!pip install autocorrect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 KB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622381 sha256=e70097e5b818562b22742f9f8ed3e03407e3b65995864d2383e817a22caa559f
  Stored in directory: /root/.cache/pip/wheels/72/b8/3b/a90246d13090e85394a8a44b78c8abf577c0766f29d6543c75
Successfully built autocorrect


In [25]:
name = 'Autocorrect'

from autocorrect import Speller
speller = Speller('en')

def predict_correct_autocorrect(inc):
    return speller(inc)
incor_data,acc,tt = check_accuracy(predict_correct_autocorrect,name)
all_results[name] = {'accuracy':acc,'time_taken':tt}

Testing Accuracy for : Autocorrect


100%|██████████| 2455/2455 [01:18<00:00, 31.31it/s] 

Model Accuracy : 0.7018
Sample Incorrections
Incorrect:Correct:Corrected
officialy : officially : official
layed : laid : played
dissobediant : disobedient : dissobediant
rigeur : rigour : rieur
ws : was : ws
pased : passed : based
liek : like : link
demographical : demographic : demographics
monolite : monolithic : monoxide
discontentment : discontent : discontentment


In [26]:
all_results

{'JamSpell': {'accuracy': 0.6676171079429736,
  'time_taken': 1.7688825130462646},
 'TextBlob': {'accuracy': 0.6167006109979634,
  'time_taken': 251.50810289382935},
 'PySpellChecker': {'accuracy': 0.7254582484725051,
  'time_taken': 118.07814455032349},
 'Autocorrect': {'accuracy': 0.7018329938900204,
  'time_taken': 78.42727327346802}}

In [27]:
pd.DataFrame(all_results)

,JamSpell,TextBlob,PySpellChecker,Autocorrect
accuracy,0.667617,0.616701,0.725458,0.701833
time_taken,1.768883,251.508103,118.078145,78.427273
